In [15]:
"""ファブリック

3次元平面ロボットアーム
"""

import numpy as np
from scipy import integrate
import sympy as sy
import matplotlib.pyplot as plt

import robot_sice_ex.sice_ex as sice_ex

def s(x_dot):
    if x_dot < 0:
        return 1
    else:
        return 0

def sigmoid(x, alpha):
    return 1 / (1 + sy.exp(-alpha * x))

In [16]:
# ee attractor
# q = sy.MatrixSymbol('q', 2, 1)
# q_d = sy.MatrixSymbol('q_d', 2, 1)
# phi = q - q_d
# x = phi
# x_norm = sy.sqrt(x[0,0]**2 + x[1,0]**2)

x = sy.MatrixSymbol('x', 2, 1)
x_norm = sy.sqrt(x[0,0]**2 + x[1,0]**2)
m_u, m_l, alpha_m, k, alpha_psi = sy.symbols('m_U, m_l, alpha_m, k, alpha_psi')



G = (m_u - m_l) * sy.exp(-(alpha_m * x_norm)**2) * sy.eye(2) + m_u * sy.eye(2)
psi_1 = k * (x_norm + 1/alpha_psi * sy.ln(1 + sy.exp(-2 * alpha_psi * x_norm)))

M_ee = G
pi_ee = -sy.Matrix([[psi_1]]).jacobian(q)

func_M_ee = sy.lambdify((x, m_u, m_l, alpha_m, k, alpha_psi), M_ee, "numpy")
func_pi_ee = sy.lambdify((x, m_u, m_l, alpha_m, k, alpha_psi), pi_ee, "numpy")


In [30]:
# obs avoidance


r, k_b, alpha_b, alpha_sig = sy.symbols('r, k_b, alpha_b, alpha_sig')
x = sy.Symbol('x')
x_dot = sy.Symbol('x_dot')

G = sigmoid(x_dot, alpha_sig) * k_b / x
L = 1/2 * x_dot * G * x_dot

M = sy.diff(L, (x_dot, 2))
psi_1b = alpha_b / (2 * x**8)


q = sy.MatrixSymbol('q', 2, 1)
q_o = sy.MatrixSymbol('q_o', 2, 1)
q_dot = sy.MatrixSymbol('q_dot', 2, 1)
q_ = q - q_o
phi = sy.sqrt(q_[0,0]**2 + q_[1,0]**2) / r - 1
x_ = phi
x_dot_ = 1/r * 1/sy.sqrt(q_[0,0]**2 + q_[1,0]**2) * (q - q_o).T * q_dot
psi_1b_ = psi_1b.subs([(x, x_)])
f = M.subs([(x, x_), (x_dot, x_dot_)]) * sy.Matrix([[psi_1b_]]).jacobian(q)

f

TypeError: Mix of Matrix and Scalar symbols

In [29]:
print(psi_1b_)

alpha_b/(2*(-1 + sqrt((q[0, 0] - q_o[0, 0])**2 + (q[1, 0] - q_o[1, 0])**2)/r)**8)


In [ ]:
print(pi_ee)

Matrix([[-k*((q[0, 0] - q_d[0, 0])/sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2) - 2*(q[0, 0] - q_d[0, 0])*exp(-2*alpha_psi*sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2))/((1 + exp(-2*alpha_psi*sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2)))*sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2))), -k*((q[1, 0] - q_d[1, 0])/sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2) - 2*(q[1, 0] - q_d[1, 0])*exp(-2*alpha_psi*sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2))/((1 + exp(-2*alpha_psi*sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2)))*sqrt((q[0, 0] - q_d[0, 0])**2 + (q[1, 0] - q_d[1, 0])**2)))]])
